In [1]:
import math
import pandas as pd
import numpy as np
import torch

### Missing data

In Pandas, using `NA` for 'missing' gives consistent results, including with strings and data/times.

> [The goal of NA is provide a “missing” indicator that can be used consistently across data types (instead of np.nan, None or pd.NaT depending on the data type).](https://pandas.pydata.org/docs/user_guide/missing_data.html#na-semantics)

In [2]:
s1 = pd.Series(["1", "2", "3", None, "5"], dtype="string")
s2 = pd.Series(["1", None, "3", None, "5"], dtype="string")

print("Equality operator:")
print(s1 == s2)
print()
print("Series.eq:")
print(pd.Series.eq(s1, s2))
print()
print("Series.equals:")
print(pd.Series.equals(s1, s2))

Equality operator:
0    True
1    <NA>
2    True
3    <NA>
4    True
dtype: boolean

Series.eq:
0    True
1    <NA>
2    True
3    <NA>
4    True
dtype: boolean

Series.equals:
False


However, this is not the same as NumPy `nan`, Python floats or `math.nan`:

In [3]:
print(f"pd.NA == pd.NA {pd.NA == pd.NA}")
print(f"np.nan == np.nan {np.nan == np.nan}")
print(f"float('nan') == float('nan') {float('nan') == float('nan')}")
print(f"math.nan == math.nan {math.nan == math.nan}")

pd.NA == pd.NA <NA>
np.nan == np.nan False
float('nan') == float('nan') False
math.nan == math.nan False


...nor, .NET `float.NaN` - though this depends on use of operator `==` or `Equals()` (see [Equals method behavior change for NaN](https://learn.microsoft.com/en-us/dotnet/core/compatibility/core-libraries/7.0/equals-nan)):

```c#
Console.WriteLine($"float.NaN == float.NaN {float.NaN == float.NaN}");
Console.WriteLine($"float.NaN.Equals(float.NaN) {float.NaN.Equals(float.NaN)}");
```

```
float.NaN == float.NaN False
```

In [4]:
print(torch.equal(torch.tensor([1, 2]), torch.tensor([1, 2])))

print(torch.equal(torch.tensor([3, torch.nan]), torch.tensor([3, torch.nan])))

True
False


With .NET `TensorPrimitives.Sum` - "If any of the values in the input is equal to <see cref="IFloatingPointIeee754{TSelf}.NaN"/>, the result is also NaN."

### Pandas Series.equals

Returns `true` if the 'same':

In [5]:
s1 = pd.Series(["1", None, "3", None, "5"], dtype="string")
s2 = pd.Series(["1", None, "3", None, "5"], dtype="string")

print("Equality operator:")
print(s1 == s2)
print()
print("Series.eq:")
print(pd.Series.eq(s1, s2))
print()
print("Series.equals:")
print(pd.Series.equals(s1, s2))

Equality operator:
0    True
1    <NA>
2    True
3    <NA>
4    True
dtype: boolean

Series.eq:
0    True
1    <NA>
2    True
3    <NA>
4    True
dtype: boolean

Series.equals:
True


## NaNumber<T>

Should we constrain to IBinaryInteger<T> so we do not end up with floating point equality problems?

So, `NaInteger<T>`?

We could have `NaFloat<T>` that uses `NaN` as `Sentinel`?

In [6]:
a = np.array([1, np.nan])
np.array_equal(a, a)

False

In [7]:
a = np.array([])
np.array_equal(a, a)

True

In [8]:
a = None
np.array_equal(a, a)

True

In [9]:
a = np.array([1, np.nan])
b = np.array([1, None])
np.array_equal(a, b)

False

In [10]:
s1 = pd.Series([], dtype="Float64")
s2 = pd.Series([], dtype="Float64")

print("Equality operator:")
print(s1 == s2)
print()
print("Series.eq:")
print(pd.Series.eq(s1, s2))
print()
print("Series.equals:")
print(pd.Series.equals(s1, s2))

Equality operator:
Series([], dtype: boolean)

Series.eq:
Series([], dtype: boolean)

Series.equals:
True


In [12]:
d = [[1, 2], [3, 4]]
df = pd.DataFrame(data=d)
df

,0,1
0,1,2
1,3,4
